In [18]:
import numpy as np
import pandas as pd

In [19]:
sgd_data = pd.read_csv('SlateGunDeaths.csv')

In [20]:
sgd_data.head()

,victimID,date,name,gender,age,ageGroup,city,state,lat,lng,url
0,1,2012-12-14,Antida Archuleta,F,20.0,3.0,Westminster,CO,39.893668,-105.072112,http://www.9news.com/rss/story.aspx?storyid=30...
1,2,2012-12-14,Charlotte Bacon,F,6.0,1.0,Newtown,CT,41.412323,-73.311424,http://www.cnn.com/2012/12/15/us/connecticut-s...
2,3,2012-12-14,Olivia Engel,F,6.0,1.0,Newtown,CT,41.412323,-73.311424,http://www.cnn.com/2012/12/15/us/connecticut-s...
3,4,2012-12-14,Ana Marquez-Greene,F,6.0,1.0,Newtown,CT,41.412323,-73.311424,http://www.cnn.com/2012/12/15/us/connecticut-s...
4,5,2012-12-14,Dylan Hockley,M,6.0,1.0,Newtown,CT,41.412323,-73.311424,http://www.cnn.com/2012/12/15/us/connecticut-s...


In [21]:
#Pre-processing data
sgd_data.columns = sgd_data.columns.str.lower()
sgd_data.date = pd.to_datetime(sgd_data.date)

In [22]:
#Filling Null Values
null_data_fill = {
    'name': 'No Name',
    'gender': 'No Gender',
    'age': 999,
    'agegroup': 9,
}
sgd_data = sgd_data.fillna(null_data_fill)
sgd_data.agegroup = sgd_data.agegroup.astype(int)
sgd_data.to_json(r'sgd_data.json',orient='records')
sgd_data.drop(['url'],axis=1,inplace=True)
sgd_data.drop([4203,10293,10576],axis=0,inplace=True)
sgd_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12067 entries, 0 to 12069
Data columns (total 10 columns):
victimid    12067 non-null int64
date        12067 non-null datetime64[ns]
name        12067 non-null object
gender      12067 non-null object
age         12067 non-null float64
agegroup    12067 non-null int64
city        12067 non-null object
state       12067 non-null object
lat         12067 non-null float64
lng         12067 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(2), object(4)
memory usage: 1.0+ MB


In [23]:
sgd_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12067 entries, 0 to 12069
Data columns (total 10 columns):
victimid    12067 non-null int64
date        12067 non-null datetime64[ns]
name        12067 non-null object
gender      12067 non-null object
age         12067 non-null float64
agegroup    12067 non-null int64
city        12067 non-null object
state       12067 non-null object
lat         12067 non-null float64
lng         12067 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(2), object(4)
memory usage: 1.0+ MB


In [24]:
#Checking if any null values
sgd_data.isna().sum()

victimid    0
date        0
name        0
gender      0
age         0
agegroup    0
city        0
state       0
lat         0
lng         0
dtype: int64

In [25]:
sgd_males_city = pd.DataFrame(sgd_data[sgd_data['gender'] == 'M'].groupby(['city', 'state'])['victimid'].count())
sgd_females_city = pd.DataFrame(sgd_data[sgd_data['gender'] == 'F'].groupby(['city', 'state'])['victimid'].count())
sgd_agegroup_1 = pd.DataFrame(sgd_data[sgd_data['agegroup'] == 1].groupby(['city', 'state'])['victimid'].count())
sgd_agegroup_2 = pd.DataFrame(sgd_data[sgd_data['agegroup'] == 2].groupby(['city', 'state'])['victimid'].count())
sgd_agegroup_3 = pd.DataFrame(sgd_data[sgd_data['agegroup'] == 3].groupby(['city', 'state'])['victimid'].count())
sgd_agegroup_9 = pd.DataFrame(sgd_data[sgd_data['agegroup'] == 9].groupby(['city', 'state'])['victimid'].count())

In [26]:
sgd_males_city.head()

victimid
city             state          
Abbeville        LA            1
                 SC            2
Abbeville County SC            1
Aberdeen         MD            1
                 NC            3

In [27]:
sgd_females_city.head()

victimid
city         state          
Acworth      GA            1
Adairsville  GA            1
Adams County CO            1
             WI            1
Adelanto     CA            1

In [28]:
sgd_agegroup_012 = pd.merge(left=sgd_agegroup_1,right=sgd_agegroup_2,how='outer',on=['city','state'],suffixes=('_agegroup_1', '_agegroup_2')).fillna(0)
sgd_agegroup_0123 = pd.merge(left=sgd_agegroup_012,right=sgd_agegroup_3,how='outer',on=['city','state'],suffixes=('_agegroup_2', '_agegroup_3')).fillna(0)
sgd_agegroup = pd.merge(left=sgd_agegroup_0123,right=sgd_agegroup_9,how='outer',on=['city','state'],suffixes=('_agegroup_3', '_agegroup_9')).fillna(0).reset_index()
sgd_agegroup.columns = ['city','state','agegroup_1','agegroup_2','agegroup_3','agegroup_9']
sgd_agegroup = sgd_agegroup.astype({'agegroup_1': 'int16', 'agegroup_2': 'int16','agegroup_3': 'int16','agegroup_9': 'int16'})
sgd_agegroup.head()

,city,state,agegroup_1,agegroup_2,agegroup_3,agegroup_9
0,Akron,OH,1,1,14,0
1,Albuquerque,NM,1,1,28,1
2,Alma,AR,2,0,2,0
3,American Fork,UT,1,0,0,0
4,Atlanta,GA,1,1,59,6


In [29]:
#Merge males and females to get total count
sgd_city = pd.merge(left=sgd_males_city,right=sgd_females_city,how='outer',on=['city','state'],suffixes=('_male', '_female')).fillna(0).reset_index()
sgd_city = sgd_city.astype({'victimid_male': 'int16', 'victimid_female': 'int16'})
sgd_city.columns = ['city','state','males','females']
sgd_city.head()

,city,state,males,females
0,Abbeville,LA,1,0
1,Abbeville,SC,2,0
2,Abbeville County,SC,1,0
3,Aberdeen,MD,1,0
4,Aberdeen,NC,3,0


In [30]:
#Merge latitude ad longitude information to the males and female counts
#Generating Frequecy data per city
sgd_city_coordinates = sgd_data[['city', 'state', 'lat', 'lng']].drop_duplicates(subset=['city', 'state'], keep='first')
sgd_age = pd.merge(left=sgd_agegroup,right=sgd_city_coordinates, how='inner', on=['city', 'state'])
sgd_city = pd.merge(left=sgd_city,right=sgd_age, how='inner', on=['city', 'state'])
sgd_city.head()

,city,state,males,females,agegroup_1,agegroup_2,agegroup_3,agegroup_9,lat,lng
0,Abbeville,LA,1,0,0,0,1,0,29.974650,-92.134292
1,Abbeville,SC,2,0,0,0,2,0,34.178172,-82.379015
2,Abbeville County,SC,1,0,0,0,1,0,34.249294,-82.473258
3,Aberdeen,MD,1,0,0,0,1,0,39.509556,-76.164120
4,Aberdeen,NC,3,0,0,1,2,0,35.131547,-79.429479


In [31]:
#Converting state codes to state names
state_details = pd.read_csv('us-states-postal-code.csv')
state_details.columns = state_details.columns.str.lower()
state_details.columns = ['statename','state']
state_details.head()

,statename,state
0,Alabama,AL
1,Alaska,AK
2,American Samoa,AS
3,Arizona,AZ
4,Arkansas,AR


In [32]:
sgd_city = pd.merge(left=sgd_city, right=state_details,on='state')
sgd_city.head()

,city,state,males,females,agegroup_1,agegroup_2,agegroup_3,agegroup_9,lat,lng,statename
0,Abbeville,LA,1,0,0,0,1,0,29.974650,-92.134292,Louisiana
1,Algiers,LA,2,0,0,0,2,0,29.951051,-90.081089,Louisiana
2,Ascension Parish,LA,1,0,0,0,1,0,30.204658,-90.889630,Louisiana
3,Avondale,LA,3,0,0,1,2,0,29.912983,-90.203687,Louisiana
4,Baker,LA,1,0,0,0,1,0,30.588243,-91.168163,Louisiana


In [33]:
sgd_city.to_csv('frequency_by_city.csv',sep=',')
sgd_city.to_json(r'frequency_by_city.json',orient='records')